<a href="https://colab.research.google.com/github/KOdin2/Machine-learning/blob/Classification/Sklearn_machine_learning_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This workbook allows a .csv file of data to be uploaded and then classified using different machine learning algorhtims from SKlearn. Different parameters of each type of algorthim is tested and the one which gives the best overall accuracy is choosen as the best results 

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
from sklearn.metrics import confusion_matrix
import csv
import matplotlib.pyplot as plt
import random
import time
import io
import pandas as pd

import progressbar
from time import sleep

Import all classifiers for testing

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn import svm
from sklearn.svm import SVC
from sklearn import neighbors, datasets
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB 
import math

In [3]:
def summary(results):

  arr = np.asarray(results)
  summary_results = []
  total = []
  name_index = 0

  for classifier in range(0, 11):

    name = arr[0,name_index]

    total_conf = np.sum(arr[:,name_index+4])
    total_count = np.sum(arr[:,name_index+5])

    total_acc = round(((total_conf[0][0]+total_conf[1][1]) / np.sum(total_conf)) * 100,2)

    total_rec, total_pur = overall_rec_and_pur(total_conf)

    metal_rec = round((total_conf[1][1] / np.sum(total_conf[1,:])) * 100, 2)
    metal_pur = round((total_conf[1][1] / np.sum(total_conf[:,1])) * 100, 2)

    not_metal_rec = round((total_conf[0][0] / np.sum(total_conf[0,:])) * 100,2)
    not_metal_pur = round((total_conf[0][0] / np.sum(total_conf[:,0])) * 100,2)

    summary_results.extend([name, total_acc, total_rec, total_pur, total_conf, total_count, metal_rec,metal_pur, not_metal_rec,not_metal_pur  ])
    name_index+=7

  #total.append(summary_results)
  return summary_results

In [4]:
#this function will calcualte the overall recovery and purity rate
def overall_rec_and_pur(con):
  a=b=rec=pur = 0
  #loop through rows to obtain overall recovery rate
  for row in con:
    total_of_row = row[a]/np.sum(row)
    if math.isnan(total_of_row):
      total_of_row=0
    rec = rec + total_of_row
    a+=1
  #loops through columns to obtain overall purity rate
  for column in con.T:
    total_of_column = column[b]/np.sum(column)
    if math.isnan(total_of_column):
      total_of_column=0
    pur = pur + total_of_column
    b+=1
    
  #return recovery rate and purity rate to 2dp
  return round((rec/a)*100,2) , round((pur/b)*100,2)

In [5]:
# this function takes in the model, x and y test and returns the result
def rates(model, x_test, y_test):
  pred = model.predict(x_test)
  conf = confusion_matrix(y_test, pred)
  rec, pur = overall_rec_and_pur(conf)
  samples = Counter(y_test)
  #return overall recovry, purity, confusion matrix and counter of smaples
  return rec, pur, conf, samples

In [19]:
class machine_learning_algorithms:

      #init runs every time we add a new acc
      #accruacy. is a class varible
      #self. is a instance varible
  def __init__(self, df_tt, df_ho): #constructor in c++

      self.df_tt = df_tt
      self.df_ho = df_ho

      self.x_train =self.df_tt.drop(['Label'], axis='columns')
      self.y_train =self.df_tt.Label
      self.x_test =self.df_ho.drop(['Label'], axis='columns')
      self.y_test =self.df_ho.Label

  def decision_tree(self):
      #create and train a decison tree model
      self.dt_model = DecisionTreeClassifier(random_state=0)
      self.dt_model.fit(self.x_train, self.y_train)

      self.dt_acc = round((self.dt_model.score(self.x_test, self.y_test))*100,2)

      self.dt_rec , self.dt_pur, self.dt_conf, self.dt_samples =  rates(self.dt_model, self.x_test, self.y_test)

      self.dt_output = ["Decision tree" ,self.dt_acc, self.dt_rec, self.dt_pur , self.dt_conf, self.dt_samples, "N/A"]

      return self.dt_output
  
  def random_forest(self):
      rf_best_acc = 0
      for number_of_trees in range(1,26):     
     
        self.rf_model = RandomForestClassifier(n_estimators=number_of_trees, random_state=0)
        self.rf_model.fit(self.x_train, self.y_train)
        self.rf_acc = round((self.rf_model.score(self.x_test, self.y_test))*100,2)

        if  self.rf_acc > rf_best_acc:
            rf_best_acc =  self.rf_acc

            self.rf_rec , self.rf_pur, self.rf_conf, self.rf_samples =  rates(self.rf_model, self.x_test, self.y_test)

            self.rf_output = ["Random forest", self.rf_acc, self.rf_rec, self.rf_pur , self.rf_conf, self.rf_samples, number_of_trees]

      return self.rf_output

  def extra_trees(self):
      et_best_acc = 0
      for number_of_trees in range(1,26):     
     
        self.et_model = ExtraTreesClassifier(n_estimators=number_of_trees, random_state=0)
        self.et_model.fit(self.x_train, self.y_train)
        self.et_acc = round((self.et_model.score(self.x_test, self.y_test))*100,2)

        if  self.et_acc > et_best_acc:
            et_best_acc =  self.et_acc

            self.et_rec , self.et_pur, self.et_conf, self.et_samples =  rates(self.et_model, self.x_test, self.y_test)

            self.et_output = ["Extra randomised trees" , self.et_acc, self.et_rec, self.et_pur , self.et_conf, self.et_samples, number_of_trees]

      return self.et_output


  def adaboost(self):
      ada_best_acc = 0
      for number_of_trees in range(10,260, 10):     
     
        self.ada_model = AdaBoostClassifier(n_estimators=number_of_trees, random_state=0)
        self.ada_model.fit(self.x_train, self.y_train)
        self.ada_acc = round((self.ada_model.score(self.x_test, self.y_test))*100,2)

        if  self.ada_acc > ada_best_acc:
            ada_best_acc =  self.ada_acc

            self.ada_rec , self.ada_pur, self.ada_conf, self.ada_samples =  rates(self.ada_model, self.x_test, self.y_test)

            self.ada_output = ["Adaboost",self.ada_acc, self.ada_rec, self.ada_pur , self.ada_conf, self.ada_samples, number_of_trees]
      
      return self.ada_output


  def svm_linear(self):
      svm_lin_best_acc = 0
      for number in range(1,26):     
     
        self.svm_lin_model = SVC(C=number, kernel='linear', random_state=0 )
        self.svm_lin_model.fit(self.x_train, self.y_train)
        self.svm_lin_acc = round((self.svm_lin_model.score(self.x_test, self.y_test))*100,2)

        if  self.svm_lin_acc > svm_lin_best_acc:
            svm_lin_best_acc =  self.svm_lin_acc

            self.svm_lin_rec , self.svm_lin_pur, self.svm_lin_conf, self.svm_lin_samples =  rates(self.svm_lin_model, self.x_test, self.y_test)

            self.svm_lin_output = ["SVM linear" ,self.svm_lin_acc, self.svm_lin_rec, self.svm_lin_pur , self.svm_lin_conf, self.svm_lin_samples, number]

      return self.svm_lin_output

  def svm_poly(self):
      svm_poly_best_acc = 0
      for number in range(1,26):     
     
        self.svm_poly_model = SVC(C=number, kernel='poly', random_state=0 )
        self.svm_poly_model.fit(self.x_train, self.y_train)
        self.svm_poly_acc = round((self.svm_poly_model.score(self.x_test, self.y_test))*100,2)

        if  self.svm_poly_acc > svm_poly_best_acc:
            svm_poly_best_acc =  self.svm_poly_acc

            self.svm_poly_rec , self.svm_poly_pur, self.svm_poly_conf, self.svm_poly_samples =  rates(self.svm_poly_model, self.x_test, self.y_test)

            self.svm_poly_output = ["SVM polynomial" ,self.svm_poly_acc, self.svm_poly_rec, self.svm_poly_pur , self.svm_poly_conf, self.svm_poly_samples, number]

      return self.svm_poly_output

  def svm_rbf(self):
      svm_rbf_best_acc = 0
      for number in range(1,26):     
     
        self.svm_rbf_model = SVC(C=number, kernel='rbf', random_state=0 )
        self.svm_rbf_model.fit(self.x_train, self.y_train)
        self.svm_rbf_acc = round((self.svm_rbf_model.score(self.x_test, self.y_test))*100,2)

        if  self.svm_rbf_acc > svm_rbf_best_acc:
            svm_rbf_best_acc =  self.svm_rbf_acc

            self.svm_rbf_rec , self.svm_rbf_pur, self.svm_rbf_conf, self.svm_rbf_samples =  rates(self.svm_rbf_model, self.x_test, self.y_test)

            self.svm_rbf_output = ["SVM rbf" , self.svm_rbf_acc, self.svm_rbf_rec, self.svm_rbf_pur , self.svm_rbf_conf, self.svm_rbf_samples, number]

      return self.svm_rbf_output


  def nb_gaussian(self):
      self.nb_gaus_model = GaussianNB()
      self.nb_gaus_model.fit(self.x_train, self.y_train)

      self.nb_gaus_acc = round((self.nb_gaus_model.score(self.x_test, self.y_test))*100,2)

      self.nb_gaus_rec , self.nb_gaus_pur, self.nb_gaus_conf, self.nb_gaus_samples =  rates(self.nb_gaus_model, self.x_test, self.y_test)

      self.nb_gaus_output = ["Naive Bayesian gaussian", self.nb_gaus_acc, self.nb_gaus_rec, self.nb_gaus_pur , self.nb_gaus_conf, self.nb_gaus_samples,"N/A"]

      return self.nb_gaus_output

  def nb_complement(self):
      self.nb_comp_model = ComplementNB()
      self.nb_comp_model.fit(self.x_train, self.y_train)

      self.nb_comp_acc = round((self.nb_comp_model.score(self.x_test, self.y_test))*100,2)

      self.nb_comp_rec , self.nb_comp_pur, self.nb_comp_conf, self.nb_comp_samples =  rates(self.nb_comp_model, self.x_test, self.y_test)

      self.nb_comp_output = ["Naive Bayesian complement" , self.nb_comp_acc, self.nb_comp_rec, self.nb_comp_pur , self.nb_comp_conf, self.nb_comp_samples, "N/A"]

      return self.nb_comp_output


  def nb_multinomial(self):
      self.nb_multi_model = MultinomialNB()
      self.nb_multi_model.fit(self.x_train, self.y_train) 

      self.nb_multi_acc = round((self.nb_multi_model.score(self.x_test, self.y_test))*100,2)

      self.nb_multi_rec , self.nb_multi_pur, self.nb_multi_conf, self.nb_multi_samples =  rates(self.nb_multi_model, self.x_test, self.y_test)

      self.nb_multi_output = ["Naive Bayesian multinomial" , self.nb_multi_acc, self.nb_multi_rec, self.nb_multi_pur , self.nb_multi_conf, self.nb_multi_samples, "N/A"]

      return self.nb_multi_output

  def k_nearest_neighbours(self):
      knn_best_acc = 0
      for n_neighbors in range(1,20):     
     
        self.knn_model =  neighbors.KNeighborsClassifier(n_neighbors)
        self.knn_model.fit(self.x_train, self.y_train)
        self.knn_acc = round((self.knn_model.score(self.x_test, self.y_test))*100,2)

        if  self.knn_acc > knn_best_acc:
            knn_best_acc =  self.knn_acc

            self.knn_rec , self.knn_pur, self.knn_conf, self.knn_samples =  rates(self.knn_model, self.x_test, self.y_test)

            self.knn_output = ["K nearrest neighbours" ,self.knn_acc, self.knn_rec, self.knn_pur , self.knn_conf, self.knn_samples, n_neighbors]
      return self.knn_output

  def run_ml_models(self):
    
      bar = progressbar.ProgressBar(maxval=11, \
          widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
      bar.start()

      self.all_results = []
      
      self.all_results.extend(self.decision_tree())
      self.all_results.extend(self.random_forest())
      self.all_results.extend(self.extra_trees())
      #print("25% Complete")
      bar.update(3)
      self.all_results.extend(self.adaboost())
      self.all_results.extend(self.svm_linear())
      self.all_results.extend(self.svm_poly())
      bar.update(3)
      #print("50% Complete")
      self.all_results.extend(self.svm_rbf())
      self.all_results.extend(self.nb_gaussian())
      self.all_results.extend(self.nb_complement())
      bar.update(3)
      #print("75% Complete")
      self.all_results.extend(self.nb_multinomial())
      self.all_results.extend(self.k_nearest_neighbours())

      bar.update(2)
      bar.finish()

      return self.all_results

In [7]:
#creates a header for the csv file for all 11 machine learning algotrhis
Header = ['Algorithm' ,'Overall accuracy', 'Overall recovery rate', 'Overall purity rate', 'Confusion matrix', 'No. samples','Varible']
csv_header = []
for number in range(0,11):
  csv_header.extend(Header)
csv_header = []
Summary_Header = ['Algorithm' ,'Overall accuracy', 'Overall recovery rate', 'Overall purity rate', 'Confusion matrix', 'No. samples', 'Metal Rec', 'Metal Pur', 'Not metal Rec', 'Not metal pur']

In [8]:
from google.colab import files
uploaded = files.upload()

Saving All colours - Br.csv to All colours - Br.csv


In [21]:
#
number_of_splits = 10
kf = KFold(n_splits=number_of_splits, shuffle=True, random_state = 0)

t0 = time.time()

#For every loaded in .csv file 
for key in uploaded:

  #display loaded in file and assign to dataframe called loaded_file
  print("File uploaded: " +str(key))
  loaded_file = pd.read_csv(io.BytesIO(uploaded[key]))
  loaded_file = loaded_file.abs()

  #Aquire output file name for loaded in data set
  output_file_name = input("Enter output file name: ")
 
  total_results = []
  results = []
  data_set = 1


  for train_index, test_index in kf.split(loaded_file):

    print("Fold " + str(data_set))
    data_set+=1

    #sleep to allow the status bar to work
    sleep(0.2)
    
    #assign the train set and holdout set
    training_data_set  = loaded_file.iloc[train_index]
    test_data_set = loaded_file.iloc[test_index]

    #Create instance of class which asssigns train and test data sets for class use
    machine_learning = machine_learning_algorithms(training_data_set, test_data_set)
    #Run run_ml_models which tests all models and obtains their resu;ts
    results.append(machine_learning.run_ml_models())


  with open('holdout_results_output_' +  output_file_name + '.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_header)
    writer.writerows(results)
    
  total_results.append(summary(results))
  total_results = np.reshape(total_results, (11, 10))

  with open('holdout_summary_' +output_file_name+ '.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(Summary_Header)
    writer.writerows(total_results)

  if output_file_name!= "test":
    files.download('holdout_results_output_'+  output_file_name + '.csv') 
    files.download('holdout_summary_' + output_file_name + '.csv') 

#print time taken for test
t1 = time.time()
total = t1-t0
print("Time Taken: " + str(total))

File uploaded: All colours - Br.csv
Enter output file name:test
Fold 1


[========================================================================] 100%


Testing of algorithms complete!

Fold 2


[========================================================================] 100%


Testing of algorithms complete!

Fold 3


...

Testing of algorithms complete!

Fold 9


[========================================================================] 100%


Testing of algorithms complete!

Fold 10


[========================================================================] 100%


Testing of algorithms complete!

TIME TAKEN: 190.97478461265564
